In [1]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
import zipfile
import matplotlib.pyplot as plt
from os.path import isfile, join
from random import sample 
import sqlalchemy as sa
import ast
from json.decoder import JSONDecodeError
import seaborn as sns
from functools import reduce
import warnings
import tqdm
from tqdm.notebook import tqdm, trange
import time
import pykakasi
import sqlite3
from thefuzz import process
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm, trange

directory = os.chdir(f'data/august/notverified')

/Users/yuikondo/opt/anaconda3/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# july and august
# ファイルを指定する。
files = [file for file in os.listdir(directory) if (file.endswith('.json'))]
tweet_files = [json.loads(line) for file in files for line in open(file, 'r')]

# tweet_file_nested = [json.loads(line) for line in open('data/0905_162649.json', 'r')]
# tweet_file = tweet_file_nested[0]

# add user and place info to each tweet
for tweet_file in tqdm(tweet_files, desc = "user and place assignment"):
    for i in range(len(tweet_file['data'])):
        for j in range(len(tweet_file['includes']['users'])):
            try:

                if tweet_file['data'][i]['author_id'] == tweet_file['includes']['users'][j]['id']:
                    #ここでuserinfoを設定
                    tweet_file['data'][i]['user_info'] = tweet_file['includes']['users'][j] 

                if 'in_reply_to_user_id' in tweet_file['data'][i]:
                        if tweet_file['data'][i]['in_reply_to_user_id'] == tweet_file['includes']['users'][j]['id']:
                            tweet_file['data'][i]['in_reply_to_user_info'] = tweet_file['includes']['users'][j]

    #             for k in range(len(tweet_file['includes']['places'])):

    #                 if 'geo' in tweet_file['data'][i]:
    #                     if 'place_id' in tweet_file['data'][i]['geo']:
    #                         if tweet_file['data'][i]['geo']['place_id'] == tweet_file['includes']['places'][k]['id']:
    #                             tweet_file['data'][i]['place_info'] = tweet_file['includes']['places'][k]

    #             if 'referenced_tweets.id.author_id' in tweet_file['data'][i]:
    #               if tweet_file['data'][i]['referenced_tweets.id.author_id'] == tweet_file['includes']['users'][j]['id']:
    #                   tweet_file['data'][i]['referenced_user_info'] = tweet_file['includes']['users'][j]

            except KeyError:
                pass

            if "referenced_tweets" in tweet_file['data'][i]:
                if "tweets" in tweet_file['includes']:
                    for k in range(len(tweet_file['includes']['tweets'])):
                        if tweet_file['data'][i]['referenced_tweets'][0]['id'] == tweet_file['includes']['tweets'][k]['id']:
                            tweet_file['data'][i]['referenced_tweets_info'] = tweet_file['includes']['tweets'][k]

#             if "referenced_tweets" in tweet_file['data'][i]:
                            if tweet_file['data'][i]['referenced_tweets_info']['author_id'] == tweet_file['includes']['users'][j]['id']:
                                 tweet_file['data'][i]['referenced_user_info'] = tweet_file['includes']['users'][j]

print("Done with user and place objects...")


tweets = [tweet_file['data'][i] for tweet_file in tweet_files for i in range(len(tweet_file['data']))]
print("Loaded a sample of", len(tweet_files), "files and", len(tweets), "tweets")

user and place assignment:   0%|          | 0/298 [00:00<?, ?it/s]

Done with user and place objects...
Loaded a sample of 298 files and 146311 tweets


## Data Frame作成

In [3]:
%%time
## info
tweet_id = []
created_at = []
conversation_id = []
text = []

# ## place
# place_full_name = []
# place_country_code = []
# place_country = []
# place_type = []
# place_bbox = []
# place_from_text = []

## Public metrics
retweet_count = []
reply_count = []
like_count = []
quote_count = []

# user info
user_id = []
username = []
user_full_name = []
user_following = []
user_followers = []
# user_location = []
user_created_at = []
user_bio = []

# in reply to user info
in_reply_to_user_id = []
in_reply_to_username = []
in_reply_to_user_full_name = []
in_reply_to_user_following = []
in_reply_to_user_followers = []
# in_reply_to_user_location = []
# in_reply_to_user_created_at = []
in_reply_to_user_bio = []

# referenced user info
referenced_user_id = []
referenced_username = []
referenced_user_full_name = []
referenced_user_following = []
referenced_user_followers = []
referenced_user_bio = []

# referenced tweets info
referenced_tweet_id = []
referenced_tweet_type = []
referenced_tweet_text = []
referenced_tweet_created_at = []
referenced_tweet_author_id = []
referenced_tweet_conversation_id = []

# # hexagon_id info
# hex_ids = []

for tweet in tqdm(tweets, desc = "data preprocessing"):
    
    '''tweet info'''
    # tweet id
    tweet_id.append(tweet['id'])
    # tweet text
    text.append(tweet['text'])
    # created_at
    created_at.append(tweet['created_at'])
    # conversation_id
    conversation_id.append(tweet['conversation_id'])
    # retweet_count
    retweet_count.append(tweet['public_metrics']['retweet_count'])
    # retweet_count
    quote_count.append(tweet['public_metrics']['quote_count'])
    # retweet_count
    like_count.append(tweet['public_metrics']['like_count'])
    # retweet_count
    reply_count.append(tweet['public_metrics']['reply_count'])
#     # hashtags
#     tags = []
#     try:
#         for hashtag in tweet['entities']['hashtags']:
#             tags.append(hashtag['tag'])
#         hashtags.append(tags)
#     except KeyError:
#         hashtags.append(np.NaN)
    
#     '''tweet place info'''
#    
    
    '''user info'''
    if 'user_info' in tweet.keys():
        # get user id
        user_id.append(tweet['user_info']['id'])
        # get user username
        username.append(tweet['user_info']['username'])
        # get user user_full_name
        user_full_name.append(tweet['user_info']['name'])
        # get user user_full_name_converted
        
        # get user friends
        user_following.append(tweet['user_info']['public_metrics']['following_count'])
        # get user followers
        user_followers.append(tweet['user_info']['public_metrics']['followers_count'])
        # get user bio
        user_bio.append(tweet['user_info']['description'])
    else:
        user_id.append(np.NaN)
        username.append(np.NaN)
        user_full_name.append(np.NaN)
        user_following.append(np.NaN)
        user_followers.append(np.NaN)
        user_bio.append(np.NaN)
        
    '''in reply to user info'''
    if 'in_reply_to_user_info' in tweet.keys():
        # get user id
        in_reply_to_user_id.append(tweet['in_reply_to_user_info']['id'])
        # get user username
        in_reply_to_username.append(tweet['in_reply_to_user_info']['username'])
        # get user user_full_name
        in_reply_to_user_full_name.append(tweet['in_reply_to_user_info']['name'])
        # get user friends
        in_reply_to_user_following.append(tweet['in_reply_to_user_info']['public_metrics']['following_count'])
        # get user followers
        in_reply_to_user_followers.append(tweet['in_reply_to_user_info']['public_metrics']['followers_count'])
#         # get user_created_at
#         in_reply_to_user_created_at.append(tweet['in_reply_to_user_info']['created_at'])
        # get user bio
        in_reply_to_user_bio.append(tweet['in_reply_to_user_info']['description'])
    else:
        in_reply_to_user_id.append(np.NaN)
        in_reply_to_username.append(np.NaN)
        in_reply_to_user_full_name.append(np.NaN)
        in_reply_to_user_following.append(np.NaN)
        in_reply_to_user_followers.append(np.NaN)
#         in_reply_to_user_created_at.append(np.NaN)
        in_reply_to_user_bio.append(np.NaN)
        

    '''referenced user info'''
    if 'referenced_user_info' in tweet.keys():
        # get user id
        referenced_user_id.append(tweet['referenced_user_info']['id'])
        # get user username
        referenced_username.append(tweet['referenced_user_info']['username'])
        # get user user_full_name
        referenced_user_full_name.append(tweet['referenced_user_info']['name'])
        # get user friends
        referenced_user_following.append(tweet['referenced_user_info']['public_metrics']['following_count'])
        # get user followers
        referenced_user_followers.append(tweet['referenced_user_info']['public_metrics']['followers_count'])
#         # get user_created_at
#         in_reply_to_user_created_at.append(tweet['in_reply_to_user_info']['created_at'])
        # get user bio
        referenced_user_bio.append(tweet['referenced_user_info']['description'])
    else:
        referenced_user_id.append(np.NaN)
        referenced_username.append(np.NaN)
        referenced_user_full_name.append(np.NaN)
        referenced_user_following.append(np.NaN)
        referenced_user_followers.append(np.NaN)
#         in_reply_to_user_created_at.append(np.NaN)
        referenced_user_bio.append(np.NaN)
    
    
    if 'referenced_tweets_info' in tweet.keys():
        # get tweet id
        referenced_tweet_id.append(tweet['referenced_tweets'][0]['id'])
        # get tweet type
        referenced_tweet_type.append(tweet['referenced_tweets'][0]['type'])
        # get tweet text
        referenced_tweet_text.append(tweet['referenced_tweets_info']['text'])
        # get tweet created_at
        referenced_tweet_created_at.append(tweet['referenced_tweets_info']['created_at'])
        # get tweet author id
        referenced_tweet_author_id.append(tweet['referenced_tweets_info']['author_id'])
        # get tweet conversation id
        referenced_tweet_conversation_id.append(tweet['referenced_tweets_info']['conversation_id'])
    else:
        referenced_tweet_id.append(np.NaN)
        referenced_tweet_type.append(np.NaN)
        referenced_tweet_text.append(np.NaN)
        referenced_tweet_created_at.append(np.NaN)
        referenced_tweet_author_id.append(np.NaN)
        referenced_tweet_conversation_id.append(np.NaN)

#     '''hexagon collection info'''
#     hex_ids.append(tweet['hex_id'])
    
# df columns
cols = ['tweet_id', 'conversation_id', 'created_at', 'text', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'user_id', 
        'username', 'user_full_name', 'user_following', 'user_followers', 'user_bio', 
        'in_reply_to_user_id', 'in_reply_to_username', 'in_reply_to_user_full_name', 'in_reply_to_user_following', 
        'in_reply_to_user_followers', 'in_reply_to_user_bio', 
        'referenced_user_id', 'referenced_username', 'referenced_user_full_name', 'referenced_user_following', 
        'referenced_user_followers', 'referenced_user_bio', 
        'referenced_tweet_id', 'referenced_tweet_type', 'referenced_tweet_text', 'referenced_tweet_created_at', 
        'referenced_tweet_author_id', 'referenced_tweet_conversation_id']

tweets_df = pd.DataFrame(list(zip(tweet_id,conversation_id,created_at,text,retweet_count, 
                                  reply_count,like_count,quote_count,user_id,username,user_full_name,user_following, 
                                  user_followers,user_bio,in_reply_to_user_id,
                                  in_reply_to_username,in_reply_to_user_full_name, in_reply_to_user_following,
                                  in_reply_to_user_followers, in_reply_to_user_bio, referenced_user_id, referenced_username, 
                                  referenced_user_full_name, referenced_user_following, referenced_user_followers, referenced_user_bio, 
                                  referenced_tweet_id, referenced_tweet_type, referenced_tweet_text, 
                                  referenced_tweet_created_at, referenced_tweet_author_id, referenced_tweet_conversation_id
                                  )), columns = cols)

print("Processed", len(tweets_df), "tweets")

tweets_df = tweets_df.drop_duplicates(subset="tweet_id")
tweets_df

data preprocessing:   0%|          | 0/146311 [00:00<?, ?it/s]

Processed 146311 tweets
CPU times: user 1.39 s, sys: 712 ms, total: 2.1 s
Wall time: 2.79 s


,tweet_id,conversation_id,created_at,text,retweet_count,reply_count,like_count,quote_count,user_id,username,...,referenced_user_full_name,referenced_user_following,referenced_user_followers,referenced_user_bio,referenced_tweet_id,referenced_tweet_type,referenced_tweet_text,referenced_tweet_created_at,referenced_tweet_author_id,referenced_tweet_conversation_id
0,1564747723387342848,1564747723387342848,2022-08-30T22:51:43.000Z,RT @NOSUKE0607: これもアレが関わったのだろうな。もう何にも信用できない日本政...,644,0,0,0,617716040,ActSludge,...,清水 潔,54.0,97240.0,フェイク入り乱れ混迷するニュースや情報をピックアップして分析・解説中。新聞社、出版社を経てテ...,1564746655445856256,retweeted,これもアレが関わったのだろうな。もう何にも信用できない日本政府。\n\n夫婦別姓調査の「保守...,2022-08-30T22:47:28.000Z,150687697,1564746655445856256
1,1564747722879422464,1564747722879422464,2022-08-30T22:51:43.000Z,RT @yamtom: 最近メディアの取材を受ける中で、選択的夫婦別姓が導入されないのも旧統...,1388,0,0,0,157117355,bigdaddy45_1969,...,山口智美,1963.0,25163.0,アメリカの田舎にくらすフェミ。文化人類学なども。,1564660093609603073,retweeted,最近メディアの取材を受ける中で、選択的夫婦別姓が導入されないのも旧統一教会の影響か的な質問を...,2022-08-30T17:03:30.000Z,52646196,1564660093609603073
2,1564747721835102208,1564747721835102208,2022-08-30T22:51:43.000Z,RT @NOSUKE0607: これもアレが関わったのだろうな。もう何にも信用できない日本政...,644,0,0,0,544012988,simanekomama,...,清水 潔,54.0,97240.0,フェイク入り乱れ混迷するニュースや情報をピックアップして分析・解説中。新聞社、出版社を経てテ...,1564746655445856256,retweeted,これもアレが関わったのだろうな。もう何にも信用できない日本政府。\n\n夫婦別姓調査の「保守...,2022-08-30T22:47:28.000Z,150687697,1564746655445856256
3,1564747703447277568,1564747703447277568,2022-08-30T22:51:38.000Z,RT @NOSUKE0607: これもアレが関わったのだろうな。もう何にも信用できない日本政...,644,0,0,0,76284158,u9est,...,清水 潔,54.0,97240.0,フェイク入り乱れ混迷するニュースや情報をピックアップして分析・解説中。新聞社、出版社を経てテ...,1564746655445856256,retweeted,これもアレが関わったのだろうな。もう何にも信用できない日本政府。\n\n夫婦別姓調査の「保守...,2022-08-30T22:47:28.000Z,150687697,1564746655445856256
4,1564747686623862784,1564747686623862784,2022-08-30T22:51:34.000Z,夫婦別姓叫んでるのはバツイチばかり。コミュ力ないと仕事だってできないだろうに。,0,0,0,0,3164779651,ipatrioticmom2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146306,1556477427794481153,1556477427794481153,2022-08-08T03:08:31.000Z,RT @tezheya: 選択的夫婦別姓と言い、戸籍制度廃止と言い、、、日本人にとって何か大...,1335,0,0,0,1124472347090538497,wCtk7LXgNC4wNiR,...,てつや,74084.0,82164.0,時事問題や最新トレンド、たまに生き方等について発信 しています。\nご相談等はお気軽にDMし...,1556475032293019648,retweeted,選択的夫婦別姓と言い、戸籍制度廃止と言い、、、日本人にとって何か大きなメリットがあるの？普通...,2022-08-08T02:59:00.000Z,171273202,1556475032293019648
146307,1556477411445067777,1556477411445067777,2022-08-08T03:08:27.000Z,RT @shoetsusato: 夫婦同姓は明治以降、日本人が護持してきた家族制度・戸籍制度...,259,0,0,0,1733471462,garaha01,...,田舎暮しの唱悦,68484.0,91679.0,北の大地で田舎暮し。凛とした日本人の心とそれを育んだ日本の伝統・文化・自然を大切に守り、歴史...,1556115263128760320,retweeted,夫婦同姓は明治以降、日本人が護持してきた家族制度・戸籍制度で殆どの日本人が同意している。\n...,2022-08-07T03:09:24.000Z,2877651816,1556115263128760320
146308,1556477286849073152,1556477286849073152,2022-08-08T03:07:57.000Z,RT @tezheya: 選択的夫婦別姓と言い、戸籍制度廃止と言い、、、日本人にとって何か大...,1335,0,0,0,1149301434799628288,JPN_nekodearu,...,てつや,74084.0,82164.0,時事問題や最新トレンド、たまに生き方等について発信 しています。\nご相談等はお気軽にDMし...,1556475032293019648,retweeted,選択的夫婦別姓と言い、戸籍制度廃止と言い、、、日本人にとって何か大きなメリットがあるの？普通...,2022-08-08T02:59:00.000Z,171273202,1556475032293019648
146309,1556477239117881345,1556477239117881345,2022-08-08T03:07:46.000Z,"RT @k__e_i: 正に""天皇制""も""女系天皇""も皇室断絶の為の造語です。加え、""夫婦別...",14,0,0,0,1175559730736988160,AXzFptbugHjCtVf,...,kei,490.0,370.0,その時その時の気分で雑多なことをつぶやこうかと。誤字多め。主に政治アカ。いいね・RTは賛同と...,1555162617160667137,retweeted,"正に""天皇制""も""女系天皇""も皇室断絶の為の造語です。加え、""夫婦別姓""も家族の崩壊から天皇...",2022-08-04T12:03:56.000Z,2928050666,1555162617160667137


In [6]:
tweets_df.to_csv("tweets_08010831_notverified.csv")


In [5]:
tweets_df["user_full_name"]

0                                                    AS
1                                            BigDaddy45
2         シマシマネコのママ　🌖🏳️‍🌈💙💛🌻（戦争反対。憲法改悪反対。原発動かすな。PCR検査を）
3                                              お仙 #戦争反対
4                                         ipatrioticmom
                              ...                      
146306                                               だい
146307                                              ガラハ
146308                                        吾輩は猫である🇯🇵
146309                                              kei
146310                                             ぽんひろ
Name: user_full_name, Length: 146311, dtype: object

In [85]:
#漢字仮名混じり文をローマ字にする（Twitterデータ）
#Loopにする

clean_name = []
for i in range(len(user_full_name)):
    kks = pykakasi.kakasi()
    result = kks.convert(user_full_name[i])
    for item in result:
        (f'原文: {item["orig"]}, ひらがな: {item["hira"]}, カナ: {item["kana"]}, ローマ字: {item["hepburn"]}')
    clean_name.append(''.join([item['hepburn'] for item in result]))

#結果をデータベースに入れる
tweets_df['clean_name'] = clean_name


In [73]:
tweets_df['clean_name']

0                            HAYASHI, Yoshi-Yuki
1                         songenshi a.k.a.ipppen
2        sentakutekifuufubesseiseidonojitsugenhe
3                                       masamune
4                                         474717
                          ...                   
19825                             inukaikatanago
19826                              sato sachino.
19827                                        T-T
19828                                    Tanny E
19829                                        aka
Name: clean_name, Length: 19830, dtype: object

In [10]:
#名前データを読み込む
# name_df = pd.read_csv("external.reference/name.gender.japan/datasets/GivenNameGenderV2.csv")
name_df = pd.read_csv("external.reference/name.gender.japan/datasets/JapaneseCelebs.csv")
name_df

clean_name = []
for i in range(len(name_df["GivenName"])):
    kks = pykakasi.kakasi()
    result = kks.convert(name_df["GivenName"][i])
    for item in result:
        (f'原文: {item["orig"]}, ひらがな: {item["hira"]}, カナ: {item["kana"]}, ローマ字: {item["hepburn"]}')
    clean_name.append(''.join([item['hepburn'] for item in result]))

#結果をデータベースに入れる
name_df['clean_name'] = clean_name
#漢字仮名混じり文をローマ字にする（名前データ）

In [70]:
name_df['clean_name']

0              ai
1              ai
2              ai
3              ai
4              ai
           ...   
12392          wa
12393        hiko
12394          ta
12395    matsurou
12396       osamu
Name: clean_name, Length: 12397, dtype: object

In [17]:
name_df_2 = name_df.drop_duplicates(subset='clean_name')
name_df_2

,GivenName,Gender,FullName,Reference,clean_name
0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai
6,あいか,f,行平 あいか,https://ja.wikipedia.org/wiki/%E8%A1%8C%E5%B9%...,aika
7,アイク,m,アイク ぬわら,https://ja.wikipedia.org/wiki/%E3%82%A2%E3%82%...,aiku
8,あいこ,f,たかはし あいこ,https://ja.wikipedia.org/wiki/%E3%81%9F%E3%81%...,aiko
11,アイザック,m,生馬 アイザック,https://ja.wikipedia.org/wiki/%E7%94%9F%E9%A6%...,aizakku
...,...,...,...,...,...
12387,昱子,f,荻 昱子,https://ja.wikipedia.org/wiki/%E8%8D%BB%E6%98%...,akirako
12388,侊彦,m,柴田 侊彦,https://ja.wikipedia.org/wiki/%E6%9F%B4%E7%94%...,hiko
12389,德馬,m,西岡 德馬,https://ja.wikipedia.org/wiki/%E8%A5%BF%E5%B2%...,uma
12390,淸子,f,泉 淸子,https://ja.wikipedia.org/wiki/%E6%B3%89%E6%B8%...,ko


In [ ]:
name_df_2

In [86]:
tweets_df['target'] = name_df_2['clean_name'].apply(lambda n: tweets_df['clean_name'].str.contains(n)).idxmax()

In [90]:
ret = pd.merge(name_df_2, tweets_df, right_on='target', left_index=True, how='left').drop('target', axis=1)

In [91]:
ret

,GivenName,Gender,FullName,Reference,clean_name_x,tweet_id,conversation_id,created_at,text,retweet_count,...,referenced_user_followers,referenced_user_bio,referenced_tweet_id,referenced_tweet_type,referenced_tweet_text,referenced_tweet_created_at,referenced_tweet_author_id,referenced_tweet_conversation_id,clean_user_full_name,clean_name_y
4.0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai,1554419935618555906,1554358056045056000,2022-08-02T10:52:47.000Z,@sabutibi777 よくある、「俺の周りで、夫婦別姓に賛成してる奴なんか見たことない」...,0.0,...,219.0,自分の名前のまま生きていく。お互い改姓しない、法的な結婚を希望します。結果的に選択的夫婦別姓...,1554366859335438336,replied_to,@subaru1000 新郎新婦側からも、縁を切ることできる良い機会です。,2022-08-02T07:21:52.000Z,1408581432163721216,1554358056045056000,474717,474717
10.0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai,1554419628063232001,1554419628063232001,2022-08-02T10:51:33.000Z,RT @sollevante2022: 表だって選択的夫婦別姓に反対している団体なんて\n神...,58.0,...,178.0,選択的夫婦別姓 ＦＩＡＴ乗り,1554081129241911296,retweeted,表だって選択的夫婦別姓に反対している団体なんて\n神道政治連盟と幸福の科学、日本会議、\nそ...,2022-08-01T12:26:29.000Z,1510893979851960322,1554081129241911296,sarii@karaaserapisuto&suisaigaie&essensuchougo...,sarii@karaaserapisuto&suisaigaie&essensuchougo...
19.0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai,1554419061546942464,1554419061546942464,2022-08-02T10:49:18.000Z,RT @GUN81288955: つか⬇️まず招待状に #選択的夫婦別姓 講演有りと告知しな...,7.0,...,1105.0,,1554382291857965057,retweeted,つか⬇️まず招待状に #選択的夫婦別姓 講演有りと告知しなよ…知らずに来て聞かされる反対の人...,2022-08-02T08:23:12.000Z,1317622986120196096,1554382291857965057,kAz(puroresu・seiji・sekaijouseisonohoka),kAz(puroresu・seiji・sekaijouseisonohoka)
22.0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai,1554418896182312960,1554418896182312960,2022-08-02T10:48:39.000Z,RT @nemuribi: 私は選択的夫婦別姓に賛成していますし、ありもしない伝統を振り翳し...,4.0,...,792.0,神社関係者です。 マイノリティの中のマイノリティと小さな活動をしたい人。全ての差別に反対しま...,1554399931854917632,retweeted,私は選択的夫婦別姓に賛成していますし、ありもしない伝統を振り翳して差別や不備を煽動・放置する...,2022-08-02T09:33:17.000Z,246655519,1554399931854917632,,
25.0,あい,f,斉川 あい,https://ja.wikipedia.org/wiki/%E6%96%89%E5%B7%...,ai,1554418764057575425,1554418764057575425,2022-08-02T10:48:07.000Z,RT @2kUow: 「選択的夫婦別姓の子供の姓」について統一することが出来てませんよね？\...,5.0,...,673.0,日本が大好きです。無言フォローすみません。中傷する人、言葉遣いが悪い人などはブロックします。...,1554338027769475072,retweeted,「選択的夫婦別姓の子供の姓」について統一することが出来てませんよね？\n\n現行法維持派は事...,2022-08-02T05:27:18.000Z,1228265017104928769,1554338027769475072,kAz(puroresu・seiji・sekaijouseisonohoka),kAz(puroresu・seiji・sekaijouseisonohoka)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18859.0,淸子,f,泉 淸子,https://ja.wikipedia.org/wiki/%E6%B3%89%E6%B8%...,ko,1554443310147969025,1554443310147969025,2022-08-02T12:25:39.000Z,RT @oriak9: 選択的夫婦別姓の話に現れる「夫婦同姓がいい人はどうするんですか！？」...,2263.0,...,532.0,逆さまの猫画像を見ると喜ぶ人です。成人済。雑食。BLGLHLなんでも好き。審神者兼マスター兼...,1554305535633391616,retweeted,選択的夫婦別姓の話に現れる「夫婦同姓がいい人はどうするんですか！？」、ハイヒール強制やめよう...,2022-08-02T03:18:11.000Z,3223432272,1554305535633391616,kookoo,kookoo
19523.0,淸子,f,泉 淸子,https://ja.wikipedia.org/wiki/%E6%B3%89%E6%B8%...,ko,1554859788441292800,1554738544228601862,2022-08-03T16:00:36.000Z,@mas__yamazaki 国政選挙のたびに選択的夫婦別姓への賛否を各党党首が問われ自民党...,0.0,...,449.0,NO WAR/統一自民党❌/世襲議員❌供託金❌/憲法が命/政教分離原則違反を許さない/統一教...,1554858650509778952,replied_to,@mas__yamazaki (SamejimaTimes)\n自民党の多くの議員が選挙で統...,2022-08-03T15:56:04.000Z,1343869287019081730,1554738544228601862,korokorokoro,korokorokoro
19783.0,淸子,f,泉 淸子,https://ja.wikipedia.org/wiki/%E6%B3%89%E6%B8%...,ko,1554841028003532800,1554841028003532800,2022-08-03T14:46:03.000Z,RT @hotaka_tsukada: 山谷先生、一年生議員の時から、統一教会系『世界日報』...,1743.0,...,25540.0,宗教社会学者。長野市出身。宗教と政治に関する問題や、カルト問題などについてつぶやきます。単著...,1554837312789368832,retweeted,山谷先生、一年生議員の時から、統一教会系『世界日報』（2001.11.25・26）で、「結婚...,2022-08-03T14:31:17.000Z,3118622494,1554829553650257920,Masa da Oldskooler,Masa da Oldskooler
19820.0,淸子,f,泉 淸子,https://ja.wikipedia.org/wiki/%E6%B3%89%E6%B8%...,ko,1554839554967736320,1554839554967736320,2022-08-03T14:40:12.000Z,RT @hotaka_tsukada: 山谷先生、一年生議員の時から、統一教会系『世界日

In [69]:
tweets_df['clean_name']

0        1657
1         340
2         187
3        1873
4           0
         ... 
19825       0
19826     380
19827       0
19828     147
19829    2206
Name: clean_name, Length: 19830, dtype: int64

In [7]:
learning = tweets_df[["username","user_full_name","user_bio"]]
learning_df = learning[1:4000].drop_duplicates()

In [8]:
learning_df

,username,user_full_name,user_bio
1,ipppen,尊厳死 a.k.a.ipppen,小児リアリスト達に捧ぐ第三次サマーオブラヴ
2,bessei_family,選択的夫婦別姓制度の実現へ,現在日本で言われている〝夫婦別姓〟とは「選択的夫婦別姓」であり、「強制的夫婦別姓」ではありま...
3,hiro_npwr,まさむね,
4,subaru1000,47×17,love all kinds of aircrafts include fixed gear...
5,mumumu_fz,いろむ,考えることを面倒くさがらずにいたい。
...,...,...,...
3994,atsusugiru___,暑すぎ無理,目についたもの気になったものなんでも。よく無言フォローします。嫌な人はブロックしてね。
3995,naya1178,ナヤ,50代シュフです\nコロナ禍で家族の命と健康を守るための情報収集アカウントになりました\nど...
3997,AXzFptbugHjCtVf,kei,すでにテレビ局は親韓や親中の反日工作機関ばかり。テレビ局は中国や韓国に乗っ取られている。移民...
3998,mitarasi_cat,みたらしだんご,日本大好き。フィギュアスケート大好き。ネコも好き。日本を好きでいてくれる国を大事にしたい＆理...


In [9]:
learning_df.to_csv('learning_data.csv')